# Aggregation with Polars

### Loading Libraries

In [1]:
# ZipFiles
import zipfile

# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd
import polars as pl
import polars.selectors as cs

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# XGBoost
import xgboost as xgb

## Loading Data

In [6]:
# Path
path = '/Users/isisromero/desktop/polars/datasets/vehicles.csv'

In [7]:
raw = pl.read_csv(path, null_values=['NA'])

In [8]:
def tweak_auto(df):
    cols = ['year', 'make', 'model', 'displ', 'cylinders', 'trany', 
            'drive', 'VClass', 'fuelType', 'barrels08', 'city08', 'highway08', 'createdOn']
    return (df
            .select(pl.col(cols))
            .with_columns(pl.col('year').cast(pl.Int16),
                          pl.col(['cylinders', 'highway08', 'city08']).cast(pl.UInt8),
                          pl.col(['displ', 'barrels08']).cast(pl.Float32),
                          pl.col(['make', 'model', 'VClass', 'drive', 'fuelType']).cast(pl.Categorical),
                          pl.col('createdOn').str.to_datetime('%a %b %d %H:%M:%S %Z %Y'),
                          is_automatic=pl.col('trany')                    
                          .str.contains('Automatic')
                          .fill_null('Automatic'),
                          num_gears=pl.col('trany')
                          .str.extract(r'(\d+)')
                          .cast(pl.UInt8)
                          .fill_null(6))
           )

In [9]:
autos = tweak_auto(raw)

In [10]:
print(autos)

shape: (48_231, 15)
┌──────┬────────────┬──────────────┬───────┬───┬───────────┬─────────────┬─────────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ highway08 ┆ createdOn   ┆ is_automati ┆ num_gears │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---       ┆ ---         ┆ c           ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ u8        ┆ datetime[μs ┆ ---         ┆ u8        │
│      ┆            ┆              ┆       ┆   ┆           ┆ ]           ┆ str         ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═══════════╪═════════════╪═════════════╪═══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 25        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆           ┆ 00:00:00    ┆             ┆           │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 14        ┆ 2013-01-01  ┆ false       ┆ 5         │
│      ┆            ┆              ┆       ┆   ┆           ┆ 00:00:00  

### Aggreations

In [12]:
col = pl.col('make')

print(sorted(att for att in dir(col) if not att.startswith('_')))

['abs', 'add', 'agg_groups', 'alias', 'all', 'and_', 'any', 'append', 'approx_n_unique', 'arccos', 'arccosh', 'arcsin', 'arcsinh', 'arctan', 'arctanh', 'arg_max', 'arg_min', 'arg_sort', 'arg_true', 'arg_unique', 'arr', 'backward_fill', 'bin', 'bitwise_and', 'bitwise_count_ones', 'bitwise_count_zeros', 'bitwise_leading_ones', 'bitwise_leading_zeros', 'bitwise_or', 'bitwise_trailing_ones', 'bitwise_trailing_zeros', 'bitwise_xor', 'bottom_k', 'bottom_k_by', 'cast', 'cat', 'cbrt', 'ceil', 'clip', 'cos', 'cosh', 'cot', 'count', 'cum_count', 'cum_max', 'cum_min', 'cum_prod', 'cum_sum', 'cumulative_eval', 'cut', 'degrees', 'deserialize', 'diff', 'dot', 'drop_nans', 'drop_nulls', 'dt', 'entropy', 'eq', 'eq_missing', 'ewm_mean', 'ewm_mean_by', 'ewm_std', 'ewm_var', 'exclude', 'exp', 'explode', 'extend_constant', 'fill_nan', 'fill_null', 'filter', 'first', 'flatten', 'floor', 'floordiv', 'forward_fill', 'from_json', 'gather', 'gather_every', 'ge', 'get', 'gt', 'has_nulls', 'hash', 'head', 'hist'

In [13]:
city = pl.col('city08')

In [14]:
# 
print(autos
      .select(mean_city=city.mean(),
              std_city=city.std(),
              var_city=city.var(),
              q99_city=city.quantile(.99)
             )
     )

shape: (1, 4)
┌───────────┬───────────┬────────────┬──────────┐
│ mean_city ┆ std_city  ┆ var_city   ┆ q99_city │
│ ---       ┆ ---       ┆ ---        ┆ ---      │
│ f64       ┆ f64       ┆ f64        ┆ f64      │
╞═══════════╪═══════════╪════════════╪══════════╡
│ 19.932533 ┆ 12.656606 ┆ 160.189684 ┆ 95.0     │
└───────────┴───────────┴────────────┴──────────┘


In [15]:
print(autos
      .with_columns(mean_city=city.mean())
     )

shape: (48_231, 16)
┌──────┬────────────┬──────────────┬───────┬───┬─────────────┬─────────────┬───────────┬───────────┐
│ year ┆ make       ┆ model        ┆ displ ┆ … ┆ createdOn   ┆ is_automati ┆ num_gears ┆ mean_city │
│ ---  ┆ ---        ┆ ---          ┆ ---   ┆   ┆ ---         ┆ c           ┆ ---       ┆ ---       │
│ i16  ┆ cat        ┆ cat          ┆ f32   ┆   ┆ datetime[μs ┆ ---         ┆ u8        ┆ f64       │
│      ┆            ┆              ┆       ┆   ┆ ]           ┆ str         ┆           ┆           │
╞══════╪════════════╪══════════════╪═══════╪═══╪═════════════╪═════════════╪═══════════╪═══════════╡
│ 1985 ┆ Alfa Romeo ┆ Spider       ┆ 2.0   ┆ … ┆ 2013-01-01  ┆ false       ┆ 5         ┆ 19.932533 │
│      ┆            ┆ Veloce 2000  ┆       ┆   ┆ 00:00:00    ┆             ┆           ┆           │
│ 1985 ┆ Ferrari    ┆ Testarossa   ┆ 4.9   ┆ … ┆ 2013-01-01  ┆ false       ┆ 5         ┆ 19.932533 │
│      ┆            ┆              ┆       ┆   ┆ 00:00:00    ┆         

### List

In [16]:
tests = pl.DataFrame({'name':['Tom', 'Sally', 'Jose'],
                      'test1':[99, 98, 95],
                      'test2':[92, None, 99],
                      'test3':[91, 93, 95],
                      'test4':[94, 92, 99]})

In [17]:
print(tests)

shape: (3, 5)
┌───────┬───────┬───────┬───────┬───────┐
│ name  ┆ test1 ┆ test2 ┆ test3 ┆ test4 │
│ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   │
│ str   ┆ i64   ┆ i64   ┆ i64   ┆ i64   │
╞═══════╪═══════╪═══════╪═══════╪═══════╡
│ Tom   ┆ 99    ┆ 92    ┆ 91    ┆ 94    │
│ Sally ┆ 98    ┆ null  ┆ 93    ┆ 92    │
│ Jose  ┆ 95    ┆ 99    ┆ 95    ┆ 99    │
└───────┴───────┴───────┴───────┴───────┘


In [18]:
print(tests
      .select(scores=pl.concat_list(cs.matches(r'test\d+')))
      .with_columns(sorted_scores=pl.col('scores').list.sort())
      .with_columns(slice_scores=pl.col('sorted_scores').list.slice(2,4))
      .with_columns(sum_scores=pl.col('slice_scores').list.sum())
     )

shape: (3, 4)
┌──────────────────┬──────────────────┬──────────────┬────────────┐
│ scores           ┆ sorted_scores    ┆ slice_scores ┆ sum_scores │
│ ---              ┆ ---              ┆ ---          ┆ ---        │
│ list[i64]        ┆ list[i64]        ┆ list[i64]    ┆ i64        │
╞══════════════════╪══════════════════╪══════════════╪════════════╡
│ [99, 92, … 94]   ┆ [91, 92, … 99]   ┆ [94, 99]     ┆ 193        │
│ [98, null, … 92] ┆ [null, 92, … 98] ┆ [93, 98]     ┆ 191        │
│ [95, 99, … 99]   ┆ [95, 95, … 99]   ┆ [99, 99]     ┆ 198        │
└──────────────────┴──────────────────┴──────────────┴────────────┘


In [20]:
# By Excluding name
print(tests.select(pl.all().exclude('name')))

shape: (3, 4)
┌───────┬───────┬───────┬───────┐
│ test1 ┆ test2 ┆ test3 ┆ test4 │
│ ---   ┆ ---   ┆ ---   ┆ ---   │
│ i64   ┆ i64   ┆ i64   ┆ i64   │
╞═══════╪═══════╪═══════╪═══════╡
│ 99    ┆ 92    ┆ 91    ┆ 94    │
│ 98    ┆ null  ┆ 93    ┆ 92    │
│ 95    ┆ 99    ┆ 95    ┆ 99    │
└───────┴───────┴───────┴───────┘


In [21]:
# Setting Difference
(tests.select(pl.all() - pl.col('name')))

InvalidOperationError: arithmetic on string and numeric not allowed, try an explicit cast first

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'select' <---
 SELECT [[(col("name")) - (col("name"))], [(col("test1")) - (col("name"))], [(col("test2")) - (col("name"))], [(col("test3")) - (col("name"))], [(col("test4")) - (col("name"))]] FROM
  DF ["name", "test1", "test2", "test3"]; PROJECT */5 COLUMNS; SELECTION: None

In [23]:
print(tests.select(cs.starts_with('test')))

shape: (3, 4)
┌───────┬───────┬───────┬───────┐
│ test1 ┆ test2 ┆ test3 ┆ test4 │
│ ---   ┆ ---   ┆ ---   ┆ ---   │
│ i64   ┆ i64   ┆ i64   ┆ i64   │
╞═══════╪═══════╪═══════╪═══════╡
│ 99    ┆ 92    ┆ 91    ┆ 94    │
│ 98    ┆ null  ┆ 93    ┆ 92    │
│ 95    ┆ 99    ┆ 95    ┆ 99    │
└───────┴───────┴───────┴───────┘


In [25]:
# Horizontal Aggregations
print(tests
      .select(scores=pl.concat_list(cs.matches(r'test\d+')))
      .with_columns(max=pl.col('scores').list.max())          
     )

shape: (3, 2)
┌──────────────────┬─────┐
│ scores           ┆ max │
│ ---              ┆ --- │
│ list[i64]        ┆ i64 │
╞══════════════════╪═════╡
│ [99, 92, … 94]   ┆ 99  │
│ [98, null, … 92] ┆ 98  │
│ [95, 99, … 99]   ┆ 99  │
└──────────────────┴─────┘


### GroupBy Operations

In [26]:
print(autos
      .group_by('make')
     )

In [28]:
# Applying Aggregation
print(autos
      .group_by('make')
      .agg(mean_city=pl.col('city08').mean())
     )

shape: (145, 2)
┌───────────────────────────────┬───────────┐
│ make                          ┆ mean_city │
│ ---                           ┆ ---       │
│ cat                           ┆ f64       │
╞═══════════════════════════════╪═══════════╡
│ Nissan                        ┆ 21.647094 │
│ Ruf Automobile Gmbh           ┆ 11.0      │
│ Grumman Olson                 ┆ 28.5      │
│ Lordstown                     ┆ 49.0      │
│ Kenyon Corporation Of America ┆ 13.0      │
│ …                             ┆ …         │
│ Saleen Performance            ┆ 11.4      │
│ Texas Coach Company           ┆ 11.0      │
│ Land Rover                    ┆ 16.180328 │
│ Polestar                      ┆ 95.96     │
│ ASC Incorporated              ┆ 14.0      │
└───────────────────────────────┴───────────┘


In [29]:
# Maintain Order
print(autos
      .group_by('make', maintain_order=True)
      .agg(mean_city=pl.col('city08').mean())
     )

shape: (145, 2)
┌──────────────────────────┬───────────┐
│ make                     ┆ mean_city │
│ ---                      ┆ ---       │
│ cat                      ┆ f64       │
╞══════════════════════════╪═══════════╡
│ Alfa Romeo               ┆ 19.5      │
│ Ferrari                  ┆ 11.911972 │
│ Dodge                    ┆ 15.466988 │
│ Subaru                   ┆ 21.130306 │
│ Toyota                   ┆ 22.4      │
│ …                        ┆ …         │
│ General Motors           ┆ 15.0      │
│ Consulier Industries Inc ┆ 18.0      │
│ Goldacre                 ┆ 16.0      │
│ Isis Imports Ltd         ┆ 16.0      │
│ PAS Inc - GMC            ┆ 14.0      │
└──────────────────────────┴───────────┘


In [30]:
# Sorting Up
print(autos
      .group_by('make')
      .agg(mean_city=pl.col('city08').mean())
      .sort('make')
     )

shape: (145, 2)
┌──────────────────────────┬───────────┐
│ make                     ┆ mean_city │
│ ---                      ┆ ---       │
│ cat                      ┆ f64       │
╞══════════════════════════╪═══════════╡
│ Alfa Romeo               ┆ 19.5      │
│ Ferrari                  ┆ 11.911972 │
│ Dodge                    ┆ 15.466988 │
│ Subaru                   ┆ 21.130306 │
│ Toyota                   ┆ 22.4      │
│ …                        ┆ …         │
│ General Motors           ┆ 15.0      │
│ Consulier Industries Inc ┆ 18.0      │
│ Goldacre                 ┆ 16.0      │
│ Isis Imports Ltd         ┆ 16.0      │
│ PAS Inc - GMC            ┆ 14.0      │
└──────────────────────────┴───────────┘


In [31]:
print(autos
      .with_columns(pl.col('make').cast(pl.Categorical('lexical')))
      .group_by('make')
      .agg(mean_city=pl.col('city08').mean())
      .sort('make')
     )

shape: (145, 2)
┌─────────────────────────────┬───────────┐
│ make                        ┆ mean_city │
│ ---                         ┆ ---       │
│ cat                         ┆ f64       │
╞═════════════════════════════╪═══════════╡
│ AM General                  ┆ 15.166667 │
│ ASC Incorporated            ┆ 14.0      │
│ Acura                       ┆ 20.011792 │
│ Alfa Romeo                  ┆ 19.5      │
│ American Motors Corporation ┆ 16.296296 │
│ …                           ┆ …         │
│ Volkswagen                  ┆ 23.39819  │
│ Volvo                       ┆ 20.355841 │
│ Wallace Environmental       ┆ 12.4375   │
│ Yugo                        ┆ 23.0      │
│ smart                       ┆ 68.421053 │
└─────────────────────────────┴───────────┘


### Multiple Aggregations

In [32]:
print(autos
      .with_columns(pl.col('make').cast(pl.Categorical('lexical')))
      .group_by('make')
      .agg(mean_city=pl.col('city08').mean(),
           mean_highway=pl.col('highway08').mean(),
           median_city=pl.col('city08').median(),
           median_highway=pl.col('highway08').median()
          )
      .sort('make')
     )

shape: (145, 5)
┌─────────────────────────────┬───────────┬──────────────┬─────────────┬────────────────┐
│ make                        ┆ mean_city ┆ mean_highway ┆ median_city ┆ median_highway │
│ ---                         ┆ ---       ┆ ---          ┆ ---         ┆ ---            │
│ cat                         ┆ f64       ┆ f64          ┆ f64         ┆ f64            │
╞═════════════════════════════╪═══════════╪══════════════╪═════════════╪════════════════╡
│ AM General                  ┆ 15.166667 ┆ 15.0         ┆ 14.5        ┆ 15.0           │
│ ASC Incorporated            ┆ 14.0      ┆ 21.0         ┆ 14.0        ┆ 21.0           │
│ Acura                       ┆ 20.011792 ┆ 26.816038    ┆ 20.0        ┆ 27.0           │
│ Alfa Romeo                  ┆ 19.5      ┆ 26.571429    ┆ 19.0        ┆ 25.0           │
│ American Motors Corporation ┆ 16.296296 ┆ 20.481481    ┆ 16.0        ┆ 20.0           │
│ …                           ┆ …         ┆ …            ┆ …           ┆ …          